In [2]:
from Scripts import WaitingtimeDistribution as wait
from lmfit import Parameters
import numpy as np

### How to set up the fit for a single Photon system

In [3]:
# Setting up the parameters, using the parameter format from lmfit
params = Parameters()
params.add("gamma_in", value=500, min=0, max=1000)
params.add("gamma_out", value=400, min=0, max=1000)
params.add("gamma_ph", value=180, min=0, max=2000, vary=False)
params.add("gamma_det", value=1e7, min=0, max=1e7, vary=False)

# Function that sets up the system-obj, whichs stores all the information needed for the calculation
def set_system(params):
    # Dictionary that determines the transitions
    rates = {   "1->3": params["gamma_out"],
                "3->1": params["gamma_in"],
                "3->4": params["gamma_ph"],
                "4->3": params["gamma_det"],
                "2->4": params["gamma_out"],
                "4->2": params["gamma_in"],
                "2->1": params["gamma_det"]}
    
    # Jumpoperators
    j1 = {  "2->1": params["gamma_det"],
            "4->3": params["gamma_det"] }
    j2 = {  "2->1": 0,  #Small trick, otherwise rates_to_matrix() does bullshit
            "3->4": params["gamma_ph"]}

    jumps = [j1, j2]

    system = wait.System(rates, jumps)
    return system

system = set_system(params)
# print(system.L)
# print(system.L0)
# print(system.jumps_matrix[0])
# print(system.jumps_matrix[1])

# Path of the timestamp data
filepath = r"/home/..."
array = np.load(filepath)
photonRatio = (len(array) / array[-1])
# setting up the fit
fitsystem = wait.FitSystem(set_system=set_system, path=filepath, photonRatio=photonRatio, doParallel=False)
result = fitsystem.fit_timestamps(params)

result

[[-4.00000e+02  1.00000e+07  5.00000e+02  0.00000e+00]
 [ 0.00000e+00 -1.00004e+07  0.00000e+00  5.00000e+02]
 [ 4.00000e+02  0.00000e+00 -6.80000e+02  1.00000e+07]
 [ 0.00000e+00  4.00000e+02  1.80000e+02 -1.00005e+07]]
[[-4.00000e+02  0.00000e+00  5.00000e+02  0.00000e+00]
 [ 0.00000e+00 -1.00004e+07  0.00000e+00  5.00000e+02]
 [ 4.00000e+02  0.00000e+00 -6.80000e+02  0.00000e+00]
 [ 0.00000e+00  4.00000e+02  0.00000e+00 -1.00005e+07]]
[[       0. 10000000.        0.        0.]
 [       0.        0.        0.        0.]
 [       0.        0.        0. 10000000.]
 [       0.        0.        0.        0.]]
[[  0.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0. 180.   0.]]


TypeError: FitSystem.__init__() missing 1 required positional argument: 'photonRatio'